1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
  
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서로 LLM에 질문과 같이 전달

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install -U langchain langchain-core langchainhub
%pip install -U pinecone

In [89]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

## chunk_size = 청크 하나를 구성하는 최대 토큰 수
## chunk_overlap = 청크들이 겹치는 토큰 수
## 예를 들어, chunk_size=1500, chunk_overlap=200이면
## 각 청크는 최대 1500 토큰까지 포함할 수 있으며,
## 각 청크는 이전 청크와 200 토큰이 겹치게 됩니다.
## 이 설정은 문서의 연속성을 유지하면서도
## 청크 간의 연결성을 높이는 데 도움이 됩니다.
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, 
  chunk_overlap=200
  )

loader = Docx2txtLoader("./tax_with_markdown.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [91]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")


In [92]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
index_name = 'tax-markdown-index'
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# ✅ 5️⃣ from_documents() 호출 시 pinecone_client 제거
database = PineconeVectorStore.from_documents(
    document_list,
    embedding=embedding,
    index_name=index_name,
)

In [104]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query = query, k=3)


In [105]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [106]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
[Identity]
- 당신은 최고의 한국 세무 전문가입니다.
- [Context]를 참고하여 사용자의 질문에 답변하세요.

[Context]
{context}

[User Question]
{input}
""")

In [107]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

# LLM과 Prompt 준비
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# database는 PineconeVectorStore 같은 Retriever로 변환 가능해야 함
retriever = database.as_retriever(search_kwargs={"k": 4})

# Retrieval + Combine 연결
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

# query -> 직장인 -> 거주자 chain 추가가

# ✅ invoke 시에는 질문만 넘김
result = rag_chain.invoke({"input": query})

print(result["answer"])


연봉 5천만원인 직장인의 소득세를 계산하려면 여러 가지 요소를 고려해야 합니다. 기본적으로 소득세는 과세표준에 의해 결정되며, 각 개인의 상황에 따라 공제 항목이 다르게 적용될 수 있습니다. 여기서는 기본적인 소득세 계산 과정을 설명드립니다.

1. **총 급여에서의 공제**:
   - 근로소득공제: 총 급여에 따라 차등 적용됩니다. 예를 들어, 연봉 5천만원인 경우, 근로소득공제는 보통 약 1,100만원 정도가 될 것입니다.
   - 공제 후 과세표준: 연봉 5천만원에서 근로소득공제 1,100만원을 뺀 3,900만원이 됩니다.

2. **소득세 계산**:
   - 소득세는 과세표준에 기본세율을 적용하여 계산합니다. 대한민국의 소득세 기본세율(2023년 기준)은 다음과 같습니다:
     - 1,200만원 이하: 6%
     - 1,200만원 ~ 4,600만원: 15%
     - 4,600만원 ~ 8,800만원: 24%
     - 8,800만원 초과: 35%
     - 3억원 초과: 38%
     - 5억원 초과: 40%
     - 10억원 초과: 45%

   - 위의 기준에 따라 과세표준 3,900만원에 대해 소득세를 계산하면:
     - 1,200만원까지: 1,200만원 × 6% = 72만원
     - 4,600만원까지의 차액(2,700만원)에 대해: 2,700만원 × 15% = 405만원
     - 총 소득세: = 72만원 + 405만원 = 477만원

3. **지방소득세**:
   - 소득세의 10%가 지방소득세로 추가됩니다. 따라서 지방소득세는 47.7만원입니다.

4. **총 소득세 및 지방소득세 합계**:
   - 524.7만원 (477만원 + 47.7만원)

위 계산에서는 기본적인 사항만을 고려하여 소득세를 계산했습니다. 개인의 세금 부담은 공제, 세액공제, 건강보험료, 국민연금 등을 포함한 다른 요소들에 의해 달라질 수 있습니다. 정확한 금액은 연말정산을 통해 산출되는 경우가 많으니, 세무 전문가와 상담하여 자신의 개인적인 상

In [114]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ["사람을 나타내는 표현 -> 거주자"]

prompt = ChatPromptTemplate.from_template(f"""

  사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
  만약 사용자의 질문이 이미 명확하다면, 그대로 반환해주세요.
                                          
  [사전] : {dictionary}

  [사용자 질문] : {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [115]:
new_question = dictionary_chain.invoke({"question": query})

In [116]:
new_question

'연봉 5천만원인 거주자의 소득세는 얼마인가요?'

In [117]:
tax_chain = {"input" : dictionary_chain} | rag_chain

In [119]:
ai_response = tax_chain.invoke({"question": query})

In [120]:
ai_response

{'input': '연봉 5천만원인 거주자의 소득세는 얼마인가요?',
 'context': [Document(id='c0964292-ab69-411f-b0a6-4873d8f4d6cd', metadata={'source': './tax_with_markdown.docx'}, page_content='제55조(세율) ①거주자의 종합소득에 대한 소득세는 해당 연도의 종합소득과세표준에 다음의 세율을 적용하여 계산한 금액(이하 “종합소득산출세액”이라 한다)을 그 세액으로 한다. <개정 2014. 1. 1., 2016. 12. 20., 2017. 12. 19., 2020. 12. 29., 2022. 12. 31.>\n\n| 종합소득 과세표준          | 세율                                         |\n\n|-------------------|--------------------------------------------|\n\n| 1,400만원 이하     | 과세표준의 6퍼센트                             |\n\n| 1,400만원 초과     5,000만원 이하     | 84만원 + (1,400만원을 초과하는 금액의 15퍼센트)  |\n\n| 5,000만원 초과   8,800만원 이하     | 624만원 + (5,000만원을 초과하는 금액의 24퍼센트) |\n\n| 8,800만원 초과 1억5천만원 이하    | 3,706만원 + (8,800만원을 초과하는 금액의 35퍼센트)|\n\n| 1억5천만원 초과 3억원 이하         | 3,706만원 + (1억5천만원을 초과하는 금액의 38퍼센트)|\n\n| 3억원 초과    5억원 이하         | 9,406만원 + (3억원을 초과하는 금액의 38퍼센트)   |\n\n| 5억원 초과      10억원 이하        | 1억 7,406만원 + (5억원을 초과하는 금액의 42퍼센트)|\n\n| 10억원 초과        | 3억 8,406만원 + (